In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights

from Model_calibration_dataset import get_cifar10_classes
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

In [2]:
# Retrieve datasets and dataloaders
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])
train_dataset, val_dataset, test_data = get_cifar10_classes(transform_train, transform_test)

# Create dataloader
batch_size=128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

# Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
# Common functions
class LabelSmoothingBCE(nn.Module):  # CrossEntropyLoss has label smoothing, BCE does not
    def __init__(self, smoothing=1.0):
        super(LabelSmoothingBCE, self).__init__()
        assert 0.0 <= smoothing < 1.0
        self.smoothing = smoothing
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, logits, target):
        with torch.no_grad():
            target = target * (1 - self.smoothing) + (self.smoothing / 2)
        return self.loss_fn(logits, target)
    

def finetune_model(model, lr, train_loader, smoothing, num_epoch=10):
    model.to(device)
    criterion = LabelSmoothingBCE(smoothing)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epoch):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epoch}, Loss: {epoch_loss:.4f}')
    return model


def evaluate_calibration(model, test_loader, label=''):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            outputs = torch.sigmoid(outputs).cpu().numpy().flatten()
            all_probs.extend(outputs)
            all_labels.extend(labels.numpy())
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    # Plot reliability curve
    prob_true, prob_pred = calibration_curve(all_labels, all_probs, n_bins=10)
    return prob_true, prob_pred, all_probs, all_labels

In [13]:
# Finetune with label smoothing and evaluate calibration 
smoothing_values = [0.0, 0.1, 0.2, 0.3]
for alpha in smoothing_values:
    print(f'Finetuning with alpha value: {alpha}')
    model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(model.fc.in_features, 1)  # binary classification
    plt.figure(figsize=(8, 6))
    results = {}
    model = finetune_model(model, 1e-3, train_loader, alpha)
    label = f'Alpha: {alpha}'
    prob_true, prob_pred, all_probs, all_labels = evaluate_calibration(model, test_loader, label)
    results[int(alpha * 10)] = {
        'prob_true': prob_true,
        'prob_pred': prob_pred,
        'all_probs': all_probs,
        'all_labels': all_labels
    }

Finetuning with alpha value: 0.0
Epoch 1/10, Loss: 0.7114
Epoch 2/10, Loss: 0.6402
Epoch 3/10, Loss: 0.6072
Epoch 4/10, Loss: 0.5882
Epoch 5/10, Loss: 0.5699
Epoch 6/10, Loss: 0.5524
Epoch 7/10, Loss: 0.5369
Epoch 8/10, Loss: 0.5245
Epoch 9/10, Loss: 0.5059
Epoch 10/10, Loss: 0.4951
Finetuning with alpha value: 0.1
Epoch 1/10, Loss: 0.7043
Epoch 2/10, Loss: 0.6601
Epoch 3/10, Loss: 0.6336
Epoch 4/10, Loss: 0.6199
Epoch 5/10, Loss: 0.6032
Epoch 6/10, Loss: 0.5922
Epoch 7/10, Loss: 0.5815
Epoch 8/10, Loss: 0.5737
Epoch 9/10, Loss: 0.5593
Epoch 10/10, Loss: 0.5492
Finetuning with alpha value: 0.2
Epoch 1/10, Loss: 0.7300
Epoch 2/10, Loss: 0.6807
Epoch 3/10, Loss: 0.6578
Epoch 4/10, Loss: 0.6434
Epoch 5/10, Loss: 0.6330
Epoch 6/10, Loss: 0.6232
Epoch 7/10, Loss: 0.6157
Epoch 8/10, Loss: 0.6084
Epoch 9/10, Loss: 0.6024
Epoch 10/10, Loss: 0.5963
Finetuning with alpha value: 0.3
Epoch 1/10, Loss: 0.7223
Epoch 2/10, Loss: 0.6883
Epoch 3/10, Loss: 0.6716
Epoch 4/10, Loss: 0.6600
Epoch 5/10, Los

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [15]:
results[int(1)]

KeyError: 1

In [14]:
# Plot
plt.figure(figsize=(8, 6))
for alpha in smoothing_values:
    prob_true = results[alpha*10]['prob_true']
    prob_pred = results[alpha*10]['prob_pred']
    plt.plot(prob_pred, prob_true, marker='o', label=f'Label Smoothing: {alpha}')
plt.plot([0, 1], [0, 1], linestyle='--', color='black')  # Draw diagonal
plt.title('Reliability with label smoothing')
plt.xlabel('Predicted probability')
plt.ylabel('Fraction of Positives')
plt.legend()
plt.show()

KeyError: 0.0

<Figure size 800x600 with 0 Axes>

In [ ]:
# Train from scratch with label smoothing
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 1)  # binary classification


In [ ]:
# Apply platt scaling on models from previous two parts and the one from part 1